In [6]:
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic

openai_api_key = os.environ['OPENAI_API_KEY']
anthropic_api_key = os.environ['ANTHROPIC_API_KEY']
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=anthropic_api_key)

In [8]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [10]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.content)

print(ai_msg.tool_calls)

messages.append(ai_msg)



[{'text': "Certainly! I can help you with those calculations using the available tools. Let's use the multiply function for the first calculation and the add function for the second one.", 'type': 'text'}, {'id': 'toolu_016pQH9Avas8vLZF8z4XEU31', 'input': {'a': 3, 'b': 12}, 'name': 'multiply', 'type': 'tool_use'}, {'id': 'toolu_014tKZv46e7N98BdBhXUkDb7', 'input': {'a': 11, 'b': 49}, 'name': 'add', 'type': 'tool_use'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'toolu_016pQH9Avas8vLZF8z4XEU31', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'toolu_014tKZv46e7N98BdBhXUkDb7', 'type': 'tool_call'}]


In [11]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)


messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content=[{'text': "Certainly! I can help you with those calculations using the available tools. Let's use the multiply function for the first calculation and the add function for the second one.", 'type': 'text'}, {'id': 'toolu_016pQH9Avas8vLZF8z4XEU31', 'input': {'a': 3, 'b': 12}, 'name': 'multiply', 'type': 'tool_use'}, {'id': 'toolu_014tKZv46e7N98BdBhXUkDb7', 'input': {'a': 11, 'b': 49}, 'name': 'add', 'type': 'tool_use'}], response_metadata={'id': 'msg_013jf3MnXGU2Rhy1g3dpeDXM', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 450, 'output_tokens': 153}}, id='run-4fc925c1-f0da-4512-be32-8a8e23e75c85-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'toolu_016pQH9Avas8vLZF8z4XEU31', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'toolu_014tKZv46e7N98BdBhXUkDb7', 'type': 'tool_call'}], usage_metadata

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content="Great! I have the results for you:\n\n1. 3 * 12 = 36\n2. 11 + 49 = 60\n\nSo, to answer your questions:\n- 3 * 12 = 36\n- 11 + 49 = 60\n\nIs there anything else you'd like me to calculate for you?", response_metadata={'id': 'msg_018VtnjZUjeYsKBCLqoGVUR3', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 664, 'output_tokens': 85}}, id='run-91db11fa-640f-45ab-bff8-f4f5e12b8a6d-0', usage_metadata={'input_tokens': 664, 'output_tokens': 85, 'total_tokens': 749})